In [ ]:
# !touch n1.txt

In [ ]:
with open('/content/n1.txt') as f:
  data=f.read()

In [ ]:
data

'The Adventure of the Three Gables\n  I don\'t think that any of my adventures with Mr. Sherlock\nHolmes opened quite so abruptly, or so dramatically, as that\nwhich I associate with The Three Gables. I had not seen Holmes\nfor some days and had no idea of the new channel into which his\nactivities had been directed. He was in a chatty mood that\nmorning, however, and had just settled me into the well-worn\nlow armchair on one side of the fire, while he had curled down\nwith his pipe in his mouth upon the opposite chair, when our\nvisitor arrived. If I had said that a mad bull had arrived it would\ngive a clearer impression of what occurred.\n  The door had flown open and a huge negro had burst into the\nroom. He would have been a comic figure if he had not been\nterrific, for he was dressed in a very loud gray check suit with a\nflowing salmon-coloured tie. His broad face and flattened nose\nwere thrust forward, as his sullen dark eyes, with a smouldering\ngleam of malice in them, tur

In [ ]:
import tensorflow as tf
import keras
import numpy as np
from tqdm import tqdm_notebook as tqdm
import nltk
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
data=data.lower()
data=re.sub(r'[\n]', ' ',data)
data=re.sub(r'[\']','',data)
data=re.sub(r'["?!]', ' ',data)
data=re.sub(r'[.,]', ' ',data)
data=re.sub(r'[:;]', ' ',data)
data=re.sub(r'[0]', ' ',data)
data=re.sub(r'[1]', ' ',data)
data=re.sub(r'[\[\]]', ' ',data)
data=re.sub(r'[\-]', ' ',data)

In [ ]:
set(data)

{' ',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [ ]:
corpus = nltk.word_tokenize(data)

In [ ]:
len(corpus)

6115

In [ ]:
vocab=set(corpus)

In [ ]:
idx2w={}
w2idx={}
for i,w in enumerate(vocab):
  idx2w[i]=w
  w2idx[w]=i
print(idx2w)
print(w2idx)
def get_window(words, window_size):
  data = []
  for i in range(len(words)):
    for j in range(i - window_size, i + window_size + 1, 1):
      if(j < 0 or j > (len(words) - 1) or j == i):
        continue
      data.append((words[i], words[j]))
  return data
def one_hot(center_word,w2idx):
  one_hot=np.zeros((len(w2idx),1))
  one_hot[w2idx[center_word]]=1
  return one_hot
def get_training_example(words, window_size, word_to_index):
  X_train, y_train = [], []
  Dataset = get_window(words, window_size)
  for centre_word, context_word in Dataset:
    X_train.append(one_hot(centre_word, word_to_index))
    y_train.append(one_hot(context_word, word_to_index))
  return X_train, y_train

{0: 'picked', 1: 'wife', 2: 'clients', 3: 'ah', 4: 'though', 5: 'obscure', 6: 'thrust', 7: 'l', 8: 'crumpled', 9: 'salmon', 10: 'london', 11: 'develop', 12: 'i', 13: 'because', 14: 'rich', 15: 'supreme', 16: 'stock', 17: 'us', 18: 'seized', 19: 'up', 20: 'derby', 21: 'fiction', 22: 'mean', 23: 'we', 24: 'mystery', 25: 'eligible', 26: 'broken', 27: 'who', 28: 'look', 29: 'profession', 30: 'am', 31: 'ruffians', 32: 'screeched', 33: 'attracts', 34: 'reference', 35: 'knowing', 36: 'wat', 37: 'amazement', 38: 'mighty', 39: 'wonderful', 40: 'four', 41: 'had', 42: 'postmark', 43: 'dainty', 44: 'gentleman', 45: 'harmless', 46: 'sense', 47: 'what', 48: 'terrific', 49: 'footman', 50: 'hiring', 51: 'cruel', 52: 'suppose', 53: 'means', 54: 'establishment', 55: 'watson', 56: 'crowd', 57: 'clatter', 58: 'broad', 59: 'youve', 60: 'lane', 61: 'mischief', 62: 'flown', 63: 'dont', 64: 'might', 65: 'examining', 66: 'noon', 67: 'gun', 68: 'haines', 69: 'yes', 70: 'doubt', 71: 'offered', 72: 'dr', 73: 'cha

In [ ]:
X_train, y_train = get_training_example(corpus, 3, w2idx)

In [ ]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
print(X_train.shape)
print(y_train.shape)

(36678, 1438, 1)
(36678, 1438, 1)


In [ ]:
X_train = np.array(X_train).reshape(1438,36678)
print(X_train.shape)
y_train = np.array(y_train).reshape(1438,36678)
print(y_train.shape)

(1438, 36678)
(1438, 36678)


In [ ]:
def Weight_Intialization(V, N):
  W1 = np.random.randn(N, V)
  b1 = np.random.randn(N, 1)
  W2 = np.random.randn(V, N)
  b2 = np.random.randn(V, 1)
  return W1, b1, W2, b2

In [ ]:
def relu(z):
  return np.maximum(0, z)

In [ ]:
def softmax(z):
  ex = tf.nn.softmax(z)
  return ex

In [ ]:
def forward_pass(x, W1, b1, W2, b2):
  Z1 = np.dot(W1, x) + b1
  Z1 = relu(Z1)
  Z2 = np.dot(W2, Z1) + b2
  ypred = softmax(Z2)
  return Z1, Z2, ypred

In [ ]:
def cost(y, y_pred, m):
  error = np.multiply(np.log(y_pred),y)+np.multiply(np.log(1-y_pred),(1-y))
  return np.squeeze((-1/m)*np.sum(error))

In [ ]:
def back_prop(W1, b1, W2, b2, Z1, Z2, y, ypred, x):
  dW1 = np.dot(relu(np.dot(W2.T, ypred - y)), x.T)
  db1 = relu(np.dot(W2.T, ypred - y))
  dW2 = np.dot(ypred - y, Z1.T)
  db2 = ypred - y
  return dW1, db1, dW2, db2

In [ ]:
def model(x, y, epoches = 10, learning_rate = 0.01):
  V = x.shape[0]
  m = x.shape[1]
  n_Hidden = 300
  W1, b1, W2, b2 = Weight_Intialization(V, n_Hidden)
  error = []
  for i in tqdm(range(epoches)):
    Z1, Z2, ypred = forward_pass(x, W1, b1, W2, b2)
    error.append(cost(y, ypred, m))
    dW1, db1, dW2, db2 = back_prop(W1, b1, W2, b2, Z1, Z2, y, ypred, x)
    W1 = W1 - learning_rate*dW1
    b1 = b1 - learning_rate*db1
    W2 = W2 - learning_rate*dW2
    b2 = b2 - learning_rate*db2
  return ypred, error, W1, W2

In [ ]:
ypred, e, W1, W2 = model(X_train, y_train, 100, 0.001)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


In [ ]:
e

In [ ]:
w=W1.T+W2